# Computer Vision, Lab 8: ROS ORB-SLAM2

Reference:
- https://github.com/appliedAI-Initiative/orb_slam_2_ros
- https://github.com/raulmur/ORB_SLAM2
- http://wiki.ros.org/
- https://arxiv.org/abs/1610.06475
- https://roboticsknowledgebase.com/wiki/state-estimation/orb-slam2-setup/
- https://medium.com/@mhamdaan/implementing-orb-slam-on-ubuntu-18-04-ros-melodic-606e668deffa
- https://medium.com/@j.zijlmans/lsd-slam-vs-orb-slam2-a-literature-based-comparison-20732df431d
- https://medium.com/@j.zijlmans/orb-slam-2052515bd84c

## What is ROS?

ROS (Robot Operating System) is an open-source for any robots. It provides the services which want (or need) to have in the robot such as hardware abstraction, low-level device control, implementation of commonly-used functionality, message-passing between processes, and package management. It provides tools and libraries for building, and running code.

## Install ROS

For Ubuntu20.04, you can install only 1 version, ROS Noetic. You can follow instruction step in the <link>[link](http://wiki.ros.org/noetic/Installation/Ubuntu)</link>. For Windows10 (not recommend) is <link>[here](http://wiki.ros.org/Installation/Windows)</link>

1. Setup your computer to accept software from packages.ros.org.

<code>
    $ sudo sh -c 'echo "deb http://packages.ros.org/ros/ubuntu $(lsb_release -sc) main" > /etc/apt/sources.list.d/ros-latest.list'
</code>

2. Set up your keys

<code>
    $ sudo apt install curl # if you haven't already installed curl
    $ curl -s https://raw.githubusercontent.com/ros/rosdistro/master/ros.asc | sudo apt-key add -
</code>

3. Install

 - Update OS
 
<code>
    $ sudo apt update
</code>

 - Desktop-Full Install: (Recommended) : Everything in Desktop plus 2D/3D simulators and 2D/3D perception packages
 
<code>
    $ sudo apt install ros-noetic-desktop-full
</code>

 - When want to install package, there are even more packages available in ROS. You can always install a specific package directly.
 
<code>
    $ sudo apt install ros-noetic-PACKAGE
</code>

For example,

<code>
    $ sudo apt install ros-noetic-slam-gmapping
</code>

4. Environment setup

You must source this script in every bash terminal you use ROS in.

<code>
    $ source /opt/ros/noetic/setup.bash
</code>

It can be convenient to automatically source this script every time a new shell is launched. These commands will do that for you.

#### Bash
<code>
    $ echo "source /opt/ros/noetic/setup.bash" >> ~/.bashrc
    $ source ~/.bashrc
</code>

5. Dependencies for building packages

<code>
    $ sudo apt install python3-rosdep python3-rosinstall python3-rosinstall-generator python3-wstool build-essential
    $ sudo apt install python3-rosdep
    $ sudo rosdep init
    $ rosdep update
</code>

### Catkin install

Sometime (or everytime), there is not catkin command function. Please install it.

<code>
    $ sudo apt-get install ros-noetic-catkin
    $ sudo apt-get install python3-catkin-tools
    $ sudo apt-get install python3-osrf-pycommon
</code>

## ORB-SLAM2

The Orb-Slam2 is followed from the <link>[paper](https://arxiv.org/pdf/1610.06475.pdf)</link>

### What is ORB-SLAM2

ORB-SLAM2 is monocular slam algorithms, where slam means Simultaneous Localization And Mapping, and monocular means that they preform slam based on a rgb image sequence (video) created by 1 camera at each time-instance.

Slam algorithms are algorithms that simultaneously tracks the movement of the camera (usually mounted onto a robot/car/etc.) and create a point cloud map of the surroundings that they passed. They create a map of the surroundings and localize them self within this map.

Monocular slam has has one big characteristic which provides it with a big pro but also a big con, it is scale independent. It cannot estimate the scale of the scenery and thus the precieved scale of the scenery will drift. This often is attempted to be fixed by trying to detect scenery that you already have been (you have traveled in a loop) and then the scale-drift can be estimated and corrected. This does bring the big pro that the algorithms work for big outdoor sceneries, small indoor sceneries and for transitions between these two.

### Feature-based vs. Direct

Monocular slam algorithms can be divided into two groups
 - feature-based methods
 - those who use direct methods
 
**Feature-based slam algorithms** take the images and within these images, they search for certain features, key-points, (for instance corners) and only use these features to estimate the location and surroundings. This means that they throw away a lot of positional valuable information from the image, but this does simplifies the whole process.

**Direct slam algorithms** do not search the image for key-points but instead use the image intensities to estimate the location and surroundings. This does mean that they use more information from the images and thus tend to be robuster and create a more detailed map of the surrounding. However they do require a lot more computational costs.

So ORB SLAM2 is *Feature-based methods*.

<img src="img/lab08-4.png" width="800"/>

ORB SLAM2 can implement in many cameras such as Monocular rgb data, stereo images (two images taken with two camera's at the same time instance), and rgb-d images (rgb with depth information).

ORB-slam2 is feature based, and uses ORB features because of the speed in which these can be extracted from images and there rotational invariance.

An overview of how the algorithm works:

<img src="img/lab08-1.png" width="800"/>

The algorithms works on three threads, a tracking thread, a local mapping thread and a loop closing thread.

### Initializing the map

To initialize the map starting by computing the relative pose between two scenes, they compute two geometrical models in parallel, one for a planar scene, a homography and one for non-planar scenes, a fundamental matrix. They then choose one of both based on a relative score of both. Using the selected model they estimate multiple motion hypotheses and en see if one is significantly better then the others, if so, a full bundle adjustment is done, otherwise the initialization starts over.

### Tracking

The tracking part localizes the camera and decides when to insert a new keyframe. Features are matched with the previous frame and the pose is optimized using motion-only bundle adjustment. The features extracted are FAST corners. (for res. till 752x480, 1000 corners should be good, for higher (KITTI 1241x376) 2000 corners works). Multiple scale-levels (factor 1.2) are used and each level is divided into a grid in which 5 corners per cell are attempted to be extracted. These FAST corners are then described using ORB. The initial pose is estimated using a constant velocity motion model. If the tracking is lost, the place recognition module kicks in and tries to re-localize itself. When there is an estimation of the pose and feature matches, the co-visibility graph of keyframes, that is maintained by the system, is used to get a local visible map. This local map consists of keyframes that share map point with the current frame, the neighbors of these keyframes and a reference keyframe which share the most map points with the current frame. Through re-projection, matches of the local map are searched on the frame and the camera pose is optimized using these matches. Finally is decided if a new Keyframe needs to be created, new keyframes are inserted very frequently to make tracking more robust. A new keyframe is created when at least 20 frames has passed from the last keyframe, and last global re-localization, the frame tracks at least 50 points of which less then 90% are point from the reference keyframe.

### Local mapping

First the new keyframe is inserted into the covisibility graph, the spanning tree linking a keyframe to the keyframe with the most points in common, and a 'bag of words' representation of the keyframe (used for data association for triangulating new points) is created.

New map points are created by triangulating ORB from connected keyframes in the covisibility graph. The unmachted ORB in a keyframe are compared with other unmatched ORB in other keyframes. The match must fulfill the epipolare constraint to be valid. To be a match, the ORB pairs are triangulated and checked if in both frames they have a positive depth, and the parallax, re projection error and scale consistency is checked. Then the match is projected to other connected keyframes to check if it is also in these.

The new map points first need to go through a test to increase the likelihood of these map points being valid. They need to be found in more than 25 % of the frames in which it is predicted to be visible and it must be observed by at least three keyframes.

Then through local bundle adjustment, the current keyframe, all keyframes connected to it through the co-visibility graph and all the map points seen by these keyframes are optimized using the keyframes that do see the map points but are not connected to the current keyframe.
Finally keyframes that are abundent are discarded to remain a certain simplicity. Keyframes from which more than 90 % of the map points can be seen by three other keyframes in the same scale-level are discarded.

### Loop closing

To detect possible loops, they check bag of words vectors of the current keyframe and its neighbors in the covisibitlity graph. The min. simularity of these bag of words vectors is taken as a benchmark and from all the keyframes with a bag of words simulatrity to the current key frame that is greater that this benchmark, all the keyframes that are allready connected to the current keyframe are removed. If three loop canditates that are consistant are detected consecutively, this loop is regarded as a serious candiddate.

For these loops, the similarity transformation is calculated (7DOF, 3 trans, 3 rot, 1 scale) RANSAC itterations are prformed to find them and these are then optimized after which more correspondences are searched and then again an optimization is preformed. If the similarity is supported by having enough inlier's, the loop is accepted.
The current keyframe pose in then adjusted and this is propagated to its neighbors and the corresponding map-points are fused. Finally a pose graph optimization is preformed over the essential graph to take out the loop closure created errors along the graph. This also corrects for scale drift.

### ORB SLAM2 Install

Follow from the <link>[link](https://github.com/appliedAI-Initiative/orb_slam_2_ros)</link> or the <link>[link2](https://github.com/raulmur/ORB_SLAM2)</link>

1. Create Workspace
<code>
    $ mkdir -p ws/src
    $ cd ws/src
    $ source /opt/ros/noetic/setup.bash
</code>

2. Download Orb slam
<code>
    $ git clone https://github.com/appliedAI-Initiative/orb_slam_2_ros.git
</code>

3. Install orb slam2
<code>
    $ cd ..
    $ rosdep install --from-paths src --ignore-src -r -y
</code>

4. Build by using catkin
<code>
    $ catkin build
</code>

### Using ORB SLAM2

1. Open another terminal to run roscore
<code>
    $ cd ws
    $ source /opt/ros/noetic/setup.bash
    $ roscore
</code>
    
2. In the old terminal lauch the orb slam. Example: Mono camera of Intel RealSense r200
<code>
    $ roslaunch orb_slam2_ros orb_slam2_r200_mono.launch
</code>

| camera | Mono | Stereo | RGBD |
|:----|:----|:----|:----|
|Intel RealSense r200 | roslaunch orb_slam2_ros orb_slam2_r200_mono.launch | roslaunch orb_slam2_ros orb_slam2_r200_stereo.launch | roslaunch orb_slam2_ros orb_slam2_r200_rgbd.launch |
|Intel RealSense d435 | roslaunch orb_slam2_ros orb_slam2_d435_mono.launch | - | roslaunch orb_slam2_ros orb_slam2_d435_rgbd.launch |
Mynteye S | roslaunch orb_slam2_ros orb_slam2_mynteye_s_mono.launch | roslaunch orb_slam2_ros orb_slam2_mynteye_s_stereo.launch | - |
Stereolabs ZED 2 | - | roslaunch orb_slam2_ros orb_slam2_zed2_stereo.launch | - |

### Do ORB SLAM2 from web camera

This process has been follow from the link: https://medium.com/@mhamdaan/implementing-orb-slam-on-ubuntu-18-04-ros-melodic-606e668deffa
In case of using USB web camera, you can use it as

1. Install USB camera and test run
<code>
    $ sudo apt install ros-melodic-usb-cam
    $ roslaunch usb_cam usb_cam-test.launch
</code>

2. Install camera_calibration modeul
<code>
    $ rosdep install camera_calibration
</code>

Once all the dependencies have been installed, you run can the camera_calibration node by giving in the required parameters. To know about each parameter and how to use camera calibration, see <link>[this page](http://wiki.ros.org/camera_calibration/Tutorials/MonocularCalibration)</link>. (You’ll also need a checkerboard to perform the calibration).
 - size: checker board size
 - square: rectangle size in meter

<code>
    $ rosrun camera_calibration cameracalibrator.py --size 8x6 --square 0.108 image:=/camera/image_raw camera:=/camera
</code>

3. convert the .ost file to a .yaml file. To do that, enter the following in the terminal:
<code>
    $ rosrun  camera_calibration_parsers convert  <filename>.ost <filename>.yaml 
</code>

Once that is over, a new .yaml file with the specified filename will be created.

4. When you run the usb_cam node, it publishes two important topics that will be subscribed by your orb_slam2_ros node. One is the /camera/image_raw and /camera/camera_info. The latter is the topic that sends your camera parameters to the orb_slam2_ros node. Therefore, you need to make your usb_cam node to publish your .yaml file parameters to that topic. To do so, enter the following commands in your terminal:

<code>
    $ roscd usb_cam
    $ cd launch
    $ sudo nano usb_cam-test.launch
</code>

The nano text editor will open up your launch file. Enter the highlighted line of code as shown in the image below: <text><param name="camera_info_url" value="file:///home/<your_username>/<your_file_address>/<your_filename>.yaml/></text>
    
<img src="img/lab08-2.png" width="800"/>
    
Save the file and close it. Your camera and its parameters are successfully setup
    
5. Setting orb_slam2_ros node (same as above)
<code>
    $ cd ~
    $ mkdir -p orbslam2_ws/src
    $ cd orbslam2_ws/src
    $ git clone https://github.com/appliedAI-Initiative/orb_slam_2_ros.git
    $ cd ..
    $ catkin_make
    $ source devel/setup.bash
</code>
    
6. Create a new .launch file in your orb_slam_2_ros/ros/launch directory and paste the following code in it and save it. After that run <code>catkin_make</code> again in your catkin workspace directory:

In [ ]:
<launch>
  <node name="orb_slam2_mono" pkg="orb_slam2_ros"
      type="orb_slam2_ros_mono" output="screen">
<param name="publish_pointcloud" type="bool" value="true" />
       <param name="publish_pose" type="bool" value="true" />
       <param name="localize_only" type="bool" value="false" />
       <param name="reset_map" type="bool" value="true" />
<!-- static parameters -->
       <param name="load_map" type="bool" value="false" />
       <param name="map_file" type="string" value="map.bin" />
       <param name="voc_file" type="string" value="$(find orb_slam2_ros)/orb_slam2/Vocabulary/ORBvoc.txt" />
<param name="pointcloud_frame_id" type="string" value="map" />
       <param name="camera_frame_id" type="string" value="camera_link" />
       <param name="min_num_kf_in_map" type="int" value="5" />
<!-- ORB parameters -->
       <param name="/ORBextractor/nFeatures" type="int" value="2000" />
       <param name="/ORBextractor/scaleFactor" type="double" value="1.2" />
       <param name="/ORBextractor/nLevels" type="int" value="8" />
       <param name="/ORBextractor/iniThFAST" type="int" value="20" />
       <param name="/ORBextractor/minThFAST" type="int" value="7" />
<!-- Camera parameters -->
       <!-- Camera frames per second -->
       <param name="camera_fps" type="int" value="30" />
       <!-- Color order of the images (0: BGR, 1: RGB. It is ignored if images are grayscale) -->
       <param name="camera_rgb_encoding" type="bool" value="true" />
        <!--If the node should wait for a camera_info topic to take the camera calibration data-->
       <param name="load_calibration_from_cam" type="bool" value="true" />
</node>
</launch>

7. Creating a node to display images

This is an important step because, when your orb_slam2_ros node runs, it will publish the live image from the camera that contains the currently found key points along with a status text. This will be published on the /debug_image topic. We cannot directly view those images, therefore we will have to create a ros node that can display those images to us.

Create a new python file inside a package and paste the following code into it. Make sure that the package has the cv_bridge dependency. If not then, you must add that dependency to your ROS package.

In [ ]:
#!/usr/bin/env python
import rospy
import cv2
from cv_bridge import CvBridge
from sensor_msgs.msg import Image
bridge = CvBridge()
def callback(data):
    frame = bridge.imgmsg_to_cv2(data, "bgr8")
    cv2.imshow('Video Feed', frame)
    cv2.waitKey(1)
    rospy.loginfo('Image feed received!')
def listener():
    rospy.init_node('vid_rec')
    #first parameter is the topic you want to subcribe sensor_msgs/Image from
    rospy.Subscriber('/orb_slam2_mono/debug_image', Image, callback)
    rospy.spin()
if __name__ == '__main__':
    listener()

Save the file and close it. Don’t forget to make your python script executable by the running the following command:
<code>
    $ chmod +x <your_python_script_name>.py
</code>

Now run catkin_make in your catkin workspace directory and then source it.

8. Running it all together. Make sure your camera is connected to your PC. Run each of the following commands in new terminals.

<code>
    $ roscore
    $ roslaunch usb_cam usb_cam-test.launch
    $ roslaunch orb_slam2_ros <your_launch_filename_you_just_created>.launch
    $ rosrun <your_package_name> <your_python_script_name>.py
    $ rviz
</code>
    
All of these must run without any errors. If there are any errors, then run rqt_graph to see if the nodes are communicating with each other. When the RVIZ window opens up, you can add tf and PointCloud2. You must also specify the correct topic name (/map_points) for RVIZ to subscribe to the PointCloud2 message from.
    
<img src="img/lab08-3.png" width="800"/>
    
Once all of that has been set up, you can now see the point clouds appearing and the TF frame of the camera link move in your RVIZ window. Feel free to explore through the GitHub page which contains the source code to use all other functionality of this amazing package. Thanks to the authors (Raul Mur-Artal, Juan D. Tardos, J. M. M. Montiel and Dorian Galvez-Lopez) who developed the orb_slam2_ros package. Now, it seems like you are all so ready to use this package for your next ROS package.

### Implement with vdo file

Try to modify the vdo stream from robot.mp4 by using process from <link>[the link](http://wiki.ros.org/video_stream_opencv)</link>